$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
$$
# Part 1: Deep Reinforcement Learning
<a id=part1></a>

In [3]:
# %% Imports

from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import util
import fid_score

cudnn.benchmark = True

#set manual seed to a constant get a consistent output
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#loading the dataset
dataset = dset.CIFAR10(root="./data", download=True,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
nc=3

dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, num_workers=0)

#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# %% Configs
ngpu = 1
# input noise dimension
nz = 100
# number of generator filters
ngf = 64
#number of discriminator filters
ndf = 64

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# %% Generator
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output

netG = Generator(ngpu).to(device)
netG.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth',map_location=torch.device('cpu')))
print(netG)

# %% Discriminator
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
#load weights to test the model 
#netD.load_state_dict(torch.load('weights/netD_epoch_24.pth'))
print(netD)


Random Seed:  5776
Files already downloaded and verified
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2

In [4]:

# %% optimizer & Loss Config

criterion = nn.BCELoss()

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
fixed_noise2 = torch.randn(49920, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

niter = 25
g_loss = []
d_loss = []


In [5]:
# %% Train Loop
#lam      = 0.4
#aug_type = 'translationX'
#util.train_loop(lam,aug_type,niter,dataloader,netG,netD,criterion,device,real_label,fake_label,nz,optimizerG,optimizerD,fixed_noise)

utidsf
# %% Train Loop
aug_type_vec   = ['translationX']
lambda_vec     = np.arange(0.1, 1.1, 0.1)
for aug_type in aug_type_vec:
    for lam in lambda_vec:
        print(aug_type, lam)
        util.createDir(aug_type,lam)
        util.train_loop(lam,aug_type,niter,dataloader,netG,netD,criterion,device,real_label,fake_label,nz,optimizerG,optimizerD,fixed_noise)

NameError: name 'utidsf' is not defined

In [7]:
# %% create folders
netG = util.load_weights(netG,path ='weights/translationX/netG_epoch_23.pth',device=device)
# create Fake Data folder
from tqdm import tqdm
torch.manual_seed(42)
for i in tqdm(range(20000)):
    fixed_noise2 = torch.randn(1, nz, 1, 1, device=device)
    fake = netG(fixed_noise2)
    vutils.save_image(fake[0,:,:,:].detach(), 'output/translationX/fake_samples/fake_sample_%03d.png' % (i), normalize=True)
    
# create Real Data folder
#for i, data in enumerate(dataloader, 0):
#    real = data[0]
#    for j in range(real.shape[0]):
#        vutils.save_image(real[j, :, :, :].detach(), 'output/real_samples/real_sample_%03d.png' % (i*real.shape[0]+j), normalize=True)

100%|██████████| 20000/20000 [02:46<00:00, 120.04it/s]


In [8]:
# %% calc FID
paths = ['output/real_samples', 'output/translationX/fake_samples/']
batch_size = 32
cuda = (device == 'cuda')
dims = 2048
model = fid_score.getInceptionModel(batch_size, cuda, dims)
m1,s1 = fid_score.getRealm1s1(paths, batch_size, cuda, dims,model)

100%|██████████| 1560/1560 [02:15<00:00, 11.55it/s]


In [1]:

# fid_score.calculate_fid_given_paths(paths, batch_size, cuda,dims,model,m1,s1)
# %% calc FID per dir
aug_type_vec   = ['translationY']
lambda_vec     = np.arange(0.1, 1.1, 0.1).round(2)
#matplotlib.use('Agg')
for aug_type in aug_type_vec:
    fid_vec = []
    for lam in lambda_vec:
        print(aug_type, lam)
        folder_name = aug_type + '_' + str(lam)  # .replace('.','p')
        paths = ['output/real_samples', 'weights/'+folder_name+'/fake_samples/']
        fid   = fid_score.calculate_fid_given_paths(paths, batch_size, cuda, dims, model, m1, s1)
        file = open("output/"+folder_name+"/result_FID.txt", "w")
        file.write(str(fid))
        file.close()
        fid_vec.append(fid)

    p = plt.plot(lambda_vec, np.array(fid_vec))
    plt.title(aug_type)
    plt.xlabel("lambda_aug")
    plt.ylabel("fid score")
    plt.savefig("output/"+aug_type+"_result_FID.png")
print("FINISH calc FID per dir")

NameError: name 'np' is not defined

In [ ]:
aug_type = 'translationY'
lam = 0.1
fid_vec = []
print(aug_type, lam)
folder_name = aug_type + '_' + str(lam)  # .replace('.','p')
paths = ['output/real_samples', 'weights/'+folder_name+'/fake_samples/']
print(paths)
fid   = fid_score.calculate_fid_given_paths(paths, batch_size, cuda, dims, model, m1, s1)
#fid = 7
print(fid)
file = open("output/"+folder_name+"/result_FID.txt", "w")
file.write(str(fid))
file.close()
fid_vec.append(fid)

translationY 0.1
['output/real_samples', 'weights/translationY_0.1/fake_samples/']


100%|██████████| 625/625 [00:53<00:00, 11.74it/s]


In [24]:
pth_path = 'weights/netG_epoch_23.pth'
netG.load_state_dict(torch.load(pth_path,map_location=torch.device(device)))
netG.state_dict()

OrderedDict([('main.0.weight',
              tensor([[[[-2.6150e-03,  4.0759e-02,  5.0097e-02, -5.0716e-03],
                        [ 1.3908e-02, -1.4891e-02,  9.6132e-03,  3.8268e-02],
                        [ 4.5625e-04,  1.8142e-02, -1.5537e-02,  4.7639e-03],
                        [ 3.4396e-02,  1.4646e-02,  3.6435e-03, -1.0926e-02]],
              
                       [[ 2.2905e-02,  1.4763e-03,  2.4345e-02, -2.4148e-02],
                        [-1.1536e-02,  5.5941e-03,  4.2937e-03,  3.5320e-02],
                        [ 3.7101e-02, -4.3300e-02,  7.9194e-04,  3.9073e-02],
                        [ 9.8261e-03,  4.7482e-03, -1.3770e-02, -6.4573e-03]],
              
                       [[-3.1728e-02,  3.6438e-02, -2.1387e-02, -1.1326e-02],
                        [-1.1868e-02, -3.5355e-03,  1.1181e-02, -3.0617e-02],
                        [-1.9670e-02, -1.3862e-02, -3.8455e-02, -5.4476e-02],
                        [ 1.8796e-02,  1.3573e-02,  2.3322e-02,  1.2863e-02]],


In [24]:
aug_type    = 'translationX'
lam         = 0.1
folder_name = aug_type + '_' + str(lam)     #.replace('.','p')
path = os.getcwd()+'/output/'+folder_name
print(path)
try:
    os.mkdir(path)
except OSError:
    print("Creation of the directory %s failed" % path)
else:
    print("Successfully created the directory %s " % path)

/home/nir.shaul/FINAL/DCGAN_CIFAR10/output/translationX_0.1
Successfully created the directory /home/nir.shaul/FINAL/DCGAN_CIFAR10/output/translationX_0.1 


**TODO**: Complete the implementation of the `forward()` method in the `ActionEntropyLoss` class.

loss =  tensor(-0.7927)


In [ ]:
display_answer(hw4.answers.part1_q3)